In [ ]:
# Please see the dataset description for additional info 
# current accuracy (balanced): see last run (this is the AraBERT implementation using ktran)

In [ ]:
import time 
print(f'Started: {time.ctime()}')

In [ ]:
!pip install -U pip
!pip install "tensorflow_gpu>=2.0.0"  # cpu: pip3 install "tensorflo
!pip install transformers==2.5.1
!pip install ktrain 

In [ ]:
# restart run time
#import os
#def restart_runtime():
#  os.kill(os.getpid(), 9)
#restart_runtime()

In [ ]:
# forked versions of eli5 and stellargraph (required by ktrain)
!pip install git+https://github.com/amaiya/eli5@tfkeras_0_10_1 
!pip install git+https://github.com/amaiya/stellargraph@no_tf_dep_082

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth',200)
pd.set_option('display.max_rows',50)
df = pd.read_excel('../input/tunisian-texts/tun.xlsx')
df['data_labels'].value_counts()

In [ ]:
df.dtypes

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df.head()

In [ ]:
"""class_names=['Positive','Negative']
df['data_labels'] = df['data_labels'].apply(lambda x: class_names.index(x))
print(len(df), '\n')
df.head()"""

In [ ]:
df_test = df.sample(frac=0.15, random_state=42)
df_train = df.drop(df_test.index)
len(df_train), len(df_test)

In [ ]:
x_train = df_train['texts'].tolist()
y_train = df_train['data_labels'].to_numpy()
x_test = df_test['texts'].tolist()
y_test = df_test['data_labels'].to_numpy()

In [ ]:
import matplotlib.pyplot as plt 
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(10,5))
tweet_len=df_train[df_train['data_labels']==1]['texts'].str.split().map(lambda x: len(x))
ax1.hist(tweet_len,color='blue')
ax1.set_title('disaster tweets')
tweet_len=df_test[df_test['data_labels']==0]['texts'].str.split().map(lambda x: len(x))
ax2.hist(tweet_len,color='CRIMSON')
ax2.set_title('Not disaster tweets')
fig.suptitle('Words in a tweet')
plt.show()

In [ ]:
import ktrain
from ktrain import text 
MODEL_NAME = 'aubmindlab/bert-base-arabertv01'
t = text.Transformer(MODEL_NAME, maxlen=90)

trn = t.preprocess_train(x_train, y_train)
val = t.preprocess_test(x_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=32)

In [ ]:
#takes long time
#learner.lr_find(show_plot=True, max_epochs=2)

In [ ]:
learner.autofit(0.1, 2)

In [ ]:
learner.validate(class_names=t.get_classes())

In [ ]:
print(learner.view_top_losses(n=5, preproc=t)) 

In [ ]:
#n = ?? # from above
#print(x_test[n])

In [ ]:
#test_text = '' # enter a review - ex. for printed above
#predictor = ktrain.get_predictor(learner.model, preproc=t)
#predictor.predict(test_text)

In [ ]:
#predictor.explain(test_text)

In [ ]:
print(f'Finished: {time.ctime()}')

#https://medium.com/analytics-vidhya/finetuning-bert-using-ktrain-for-disaster-tweets-classification-18f64a50910b